#### **Fundamentos de Bancos de Dados Relacionais e NoSQL**

#### **Conteúdo - Bases e Notebook da aula**

https://github.com/FIAP/Pos_Tech_DTAT/tree/main/Fase%203

#### **Importação de pacotes, bibliotecas e funções (def)**

In [2]:
#!pip uninstall boto3 -y
!pip uninstall botocore -y
# !pip install boto3==1.40.26 
# !pip install botocore==1.40.26
# # !pip install s3fs



Found existing installation: botocore 1.40.26
Uninstalling botocore-1.40.26:
  Successfully uninstalled botocore-1.40.26


In [3]:
#!pip install boto3==1.40.26 
!pip install botocore==1.40.26

  Using cached botocore-1.40.26-py3-none-any.whl.metadata (5.7 kB)
Using cached botocore-1.40.26-py3-none-any.whl (14.0 MB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.12.3 requires botocore<1.34.70,>=1.34.41, but you have botocore 1.40.26 which is incompatible.


In [4]:
import boto3, botocore
print("boto3:", boto3.__version__)
print("botocore:", botocore.__version__)


boto3: 1.40.26
botocore: 1.40.26


In [5]:
# Importar biblioteca completa
import boto3
import pandas as pd
import os
import plotly.express as px
import requests
import botocore
import psycopg2
import numpy as np
import duckdb
import mongomock
import fakeredis
import uuid
import json

# Importar função especifica de um módulo
from botocore.exceptions import BotoCoreError, ClientError
from sqlalchemy import create_engine, text, inspect
from dotenv import load_dotenv
from io import StringIO
from datetime import datetime
from pprint import pprint
from astrapy import DataAPIClient
# from cassandra.cluster import Cluster
# from cassandra.auth import PlainTextAuthProvider

#### **Testar conexão AWS via Python**

In [6]:
# Validar conexão
try:
    session = boto3.Session(profile_name="default")
    sts = session.client("sts")
    identity = sts.get_caller_identity()
    print("✅ Conectado à conta\n")
    print("UserId:", identity["UserId"])
    print("Account:", identity["Account"])
    print("Arn:", identity["Arn"])

except (BotoCoreError, ClientError) as e:
    print("❌ Erro ao conectar à AWS. Verifique suas credenciais e tente novamente.")
    print("Detalhes do erro:", e)


✅ Conectado à conta

UserId: AROAZI2LFHB2TERRDYNA4:user4377774=maradinidiego16@gmail.com
Account: 637423401077
Arn: arn:aws:sts::637423401077:assumed-role/voclabs/user4377774=maradinidiego16@gmail.com


##### **Conectar ao PostgreSQL via RDS + Executar Comandos SQL**

In [7]:
%pip install StringIO

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement StringIO (from versions: none)
ERROR: No matching distribution found for StringIO


In [8]:

load_dotenv("Users\dmaradini\.aws\credentials")

aws_access_key_id       = os.getenv("aws_access_key_id")
aws_secret_access_key   = os.getenv("aws_secret_access_key")
aws_session_token       = os.getenv("aws_session_token")  

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\dmaradini\AppData\Local\Temp\ipykernel_20136\2708774549.py:1: SyntaxWarning: invalid escape sequence '\d'
  load_dotenv("Users\dmaradini\.aws\credentials")


In [10]:
import pandas as pd
import boto3
import numpy as npfrp
from datetime import datetime
from io import StringIO
from botocore.exceptions import ClientError


session = boto3.Session(
    aws_access_key_id       =   aws_access_key_id
    ,aws_secret_access_key  =   aws_secret_access_key
    ,aws_session_token      =   aws_session_token
)



bucket_name = "techchallange-637423401077"
s3_prefix = "raw"



print("Conexão estabelecida.")

s3 = session.client('s3')
region = s3.meta.region_name or "us-east-1"

try:
    s3.head_bucket(Bucket=bucket_name)
    print(f"Bucket '{bucket_name}' já existe e está acessível.")
except ClientError as e:
    error_code = e.response['Error']['Code']
    if error_code in ("404", "NoSuchBucket"):
        print(f"Bucket '{bucket_name}' não existe, criando...\n")
        if region == "us-east-1":
            s3.create_bucket(Bucket=bucket_name)
            
        else:
            s3.create_bucket(
                Bucket=bucket_name,
                CreateBucketConfiguration={'LocationConstraint': region}
            )
        print(f"Bucket '{bucket_name}' criado com sucesso.\n")
    else:
        print(f"Erro ao acessar o bucket: {e}")
        raise


Conexão estabelecida.
Bucket 'techchallange-637423401077' não existe, criando...

Bucket 'techchallange-637423401077' criado com sucesso.



In [35]:
import pandas as pd
import boto3
from io import BytesIO, StringIO
from zipfile import ZipFile
import requests

bucket_name = "techchallange-637423401077"
s3_prefix = "raw"


s3 = boto3.client("s3")  # assume que suas credenciais AWS estão no ambiente

# URLs dos arquivos zipados do IBGE
urls = {
    "PNAD_COVID_052020": "https://ftp.ibge.gov.br/Trabalho_e_Rendimento/Pesquisa_Nacional_por_Amostra_de_Domicilios_PNAD_COVID19/Microdados/Dados/PNAD_COVID_052020.zip"
    ,"PNAD_COVID_062020": "https://ftp.ibge.gov.br/Trabalho_e_Rendimento/Pesquisa_Nacional_por_Amostra_de_Domicilios_PNAD_COVID19/Microdados/Dados/PNAD_COVID_062020.zip"
    ,"PNAD_COVID_072020": "https://ftp.ibge.gov.br/Trabalho_e_Rendimento/Pesquisa_Nacional_por_Amostra_de_Domicilios_PNAD_COVID19/Microdados/Dados/PNAD_COVID_072020.zip"
    ,"Dicionario_PNAD_COVID_052020":"https://ftp.ibge.gov.br/Trabalho_e_Rendimento/Pesquisa_Nacional_por_Amostra_de_Domicilios_PNAD_COVID19/Microdados/Documentacao/Dicionario_PNAD_COVID_052020_20220621.xls"
    ,"Dicionario_PNAD_COVID_062020":"https://ftp.ibge.gov.br/Trabalho_e_Rendimento/Pesquisa_Nacional_por_Amostra_de_Domicilios_PNAD_COVID19/Microdados/Documentacao/Dicionario_PNAD_COVID_062020_20220621.xls"
    ,"Dicionario_PNAD_COVID_072020":"https://ftp.ibge.gov.br/Trabalho_e_Rendimento/Pesquisa_Nacional_por_Amostra_de_Domicilios_PNAD_COVID19/Microdados/Documentacao/Dicionario_PNAD_COVID_072020_20220621.xls"
}

for tabela, url in urls.items():
    print(f"Processando {tabela}...")

    # Baixar o arquivo zip
    r = requests.get(url)
    r.raise_for_status()

    # Ler o CSV dentro do ZIP
    with ZipFile(BytesIO(r.content)) as z:
        # Assumindo que tem apenas 1 CSV dentro do zip
        csv_name = z.namelist()[0]
        with z.open(csv_name) as f:
            df = pd.read_csv(f, sep=';', encoding='latin1')  # ajuste separador e encoding

    print(f"{tabela} salva no S3 em s3://{bucket_name}/{s3_key}")
    # Exportar para S3
    csv_buffer = StringIO()
    df.to_csv(csv_buffer, index=False)
    s3_key = f"{s3_prefix}/{tabela}.csv"

    s3.put_object(
        Bucket=bucket_name,
        Key=s3_key,
        Body=csv_buffer.getvalue()
    )

    print(f"{tabela} salva no S3 em s3://{bucket_name}/{s3_key}")

print("Exportação concluída com sucesso.")


Processando PNAD_COVID_052020...
PNAD_COVID_052020 salva no S3 em s3://techchallange-637423401077/raw/PNAD_COVID_072020.csv
PNAD_COVID_052020 salva no S3 em s3://techchallange-637423401077/raw/PNAD_COVID_052020.csv
Processando PNAD_COVID_062020...
PNAD_COVID_062020 salva no S3 em s3://techchallange-637423401077/raw/PNAD_COVID_052020.csv
PNAD_COVID_062020 salva no S3 em s3://techchallange-637423401077/raw/PNAD_COVID_062020.csv
Processando PNAD_COVID_072020...
PNAD_COVID_072020 salva no S3 em s3://techchallange-637423401077/raw/PNAD_COVID_062020.csv
PNAD_COVID_072020 salva no S3 em s3://techchallange-637423401077/raw/PNAD_COVID_072020.csv
Processando Dicionario_PNAD_COVID_052020...
Dicionario_PNAD_COVID_052020 salva no S3 em s3://techchallange-637423401077/raw/PNAD_COVID_072020.csv
Dicionario_PNAD_COVID_052020 salva no S3 em s3://techchallange-637423401077/raw/Dicionario_PNAD_COVID_052020.csv
Processando Dicionario_PNAD_COVID_062020...
Dicionario_PNAD_COVID_062020 salva no S3 em s3://tec

In [36]:
import boto3

session = boto3.Session(profile_name="default")
s3 = session.client("s3")
print("Consegui conectar no S3!")


Consegui conectar no S3!


In [ ]:
!pip install s3fs


  Using cached botocore-1.34.69-py3-none-any.whl.metadata (5.7 kB)
Using cached botocore-1.34.69-py3-none-any.whl (12.0 MB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.40.26
    Uninstalling botocore-1.40.26:
      Successfully uninstalled botocore-1.40.26


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
boto3 1.40.26 requires botocore<1.41.0,>=1.40.26, but you have botocore 1.34.69 which is incompatible.
s3transfer 0.13.1 requires botocore<2.0a.0,>=1.37.4, but you have botocore 1.34.69 which is incompatible.


In [ ]:
import pandas as pd
from io import StringIO

RAW_PREFIX      = 'https://aula-data-lake-637423401077.s3.us-east-1.amazonaws.com/raw/'
SILVER_PREFIX   = 's3://aula-data-lake-637423401077/silver/'
GOLD_PREFIX     = 's3://aula-data-lake-637423401077/gold/'


tabelas = ["dim_cliente", "dim_produto", "fato_pedidos"]

clientes = pd.read_csv(RAW_PREFIX + 'dim_cliente.csv')
produtos = pd.read_csv(RAW_PREFIX + 'dim_produto.csv')
pedidos  = pd.read_csv(RAW_PREFIX + 'fato_pedidos.csv')



#1 .Silver layer: Leitura e limpeza dos Dados

for df in [clientes,produtos,pedidos]:
    df.columns = [col.lower() for col in df.columns]


#Limpeza básica 

clientes = clientes.drop_duplicates().dropna(subset=['id_cliente'])
produtos = produtos.drop_duplicates().dropna(subset=['id_produto'])
pedidos  = pedidos.drop_duplicates().dropna(subset=['id_pedido','id_cliente','id_produto'])


# Padronização das datas
if 'data_pedido' in pedidos.columns:
    pedidos['data_pedido'] = pd.to_datetime(pedidos['data_pedido'],errors='coerce')

#Salva na camada silver em parquet no S3

clientes.to_parquet(SILVER_PREFIX + 'clientes/',index=False)
produtos.to_parquet(SILVER_PREFIX + 'produtos/',index=False)
pedidos.to_parquet(SILVER_PREFIX  + 'pedidos/',index=False)

In [ ]:
pedidos_clientes = pd.merge(
    pedidos,
    clientes,
    on="id_cliente",
    how="inner"      
)

In [ ]:
pedidos_clientes_produtos = pd.merge(
    pedidos_clientes,
    produtos,
    on="id_produto",
    how="inner"
)

pedidos_ordenado = pedidos_clientes_produtos.sort_values("data_pedido",ascending=True)


In [ ]:
pedidos_clientes = pedidos_clientes_produtos[['nome','data_pedido','quantidade','valor_total','nome_produto','preco']]


,nome,data_pedido,quantidade,valor_total,nome_produto,preco
0,Luigi Porto,2024-12-02,5,2992.85,Ratione Ad,972.17
2,Luigi Porto,2025-01-27,3,2806.08,Delectus Vel,786.77
4,Ana Clara Nascimento,2024-07-20,3,1387.95,Doloribus Culpa,997.35
5,Bianca Fernandes,2025-03-02,1,463.70,Quo Culpa,220.50
6,Luigi Porto,2024-08-21,1,542.26,Ipsam Eum,254.16
...,...,...,...,...,...,...
91,Francisco Rezende,2025-04-24,1,950.12,Sunt Voluptas,807.00
92,Lucas Nascimento,2025-02-05,5,804.30,Quis At,227.42
93,Srta. Ana Carolina Ferreira,2024-11-15,1,568.33,Fugiat Omnis,236.65
96,Isis da Mata,2025-03-15,2,1930.14,Ducimus Molestias,434.15


In [ ]:
pedidos_clientes.to_parquet(GOLD_PREFIX + 'pedidos_clientes/',index=False)